In [8]:
import pandas
import pandas as pd
import numpy as np
import sys
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tqdm import tqdm

deepctr_path = '/root/linghui/rec-fairness/'
sys.path.append(deepctr_path)
import deepctr
from deepctr.models import DeepFEFM
from deepctr.feature_column import SparseFeat, VarLenSparseFeat, get_feature_names

saved_model_path = '/data/linghui/saved_model/deepfm-ml-1m'

In [9]:
saved_data_path = '/data/linghui/ml-1m/processed_data/'
processed_data = saved_data_path + 'ml-1m-negrate-1.csv'

raw_data = pd.read_csv(processed_data,index_col=0)
raw_data

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
0,0,189,1,978300760,One Flew Over the Cuckoo's Nest (1975),"[1, 0, 0, 0, 0, 0]",0,0,2,1588
1,0,3374,1,978302109,James and the Giant Peach (1996),"[2, 3, 4, 0, 0, 0]",0,0,2,1588
2,0,3615,1,978301968,My Fair Lady (1964),"[4, 5, 0, 0, 0, 0]",0,0,2,1588
3,0,2503,1,978300275,Erin Brockovich (2000),"[1, 0, 0, 0, 0, 0]",0,0,2,1588
4,0,1374,1,978824291,"Bug's Life, A (1998)","[2, 3, 6, 0, 0, 0]",0,0,2,1588
...,...,...,...,...,...,...,...,...,...,...
2000413,5602,1382,0,0,Glen or Glenda (1953),"[1, 0, 0, 0, 0, 0]",1,2,17,466
2000414,5602,159,0,0,Two or Three Things I Know About Her (1966),"[1, 0, 0, 0, 0, 0]",1,2,17,466
2000415,5602,2564,0,0,That's Life! (1986),"[1, 0, 0, 0, 0, 0]",1,2,17,466
2000416,5602,2791,0,0,"Man with the Golden Arm, The (1955)","[1, 0, 0, 0, 0, 0]",1,2,17,466


In [10]:
train_data , test_data = train_test_split(raw_data,test_size=0.2)

In [15]:
train_data.to_csv(saved_data_path + 'train_data.csv')
test_data.to_csv(saved_data_path + 'test_data.csv')

In [16]:
train_data_path = saved_data_path + 'train_data.csv' 
test_data_path = saved_data_path + 'test_data.csv'

train_data = pd.read_csv(train_data_path,index_col=0)
train_data

/opt/conda/lib/python3.8/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zipcode
550763,2658,96,1,1042575453,"Doors, The (1991)","[1, 4, 0, 0, 0, 0]",1,1,15,1612
645964,3208,77,1,965806436,Willy Wonka and the Chocolate Factory (1971),"[8, 3, 6, 9, 0, 0]",1,1,15,2561
570742,2781,502,1,966997407,Apollo 13 (1995),"[1, 0, 0, 0, 0, 0]",0,3,18,1091
1256036,622,93,0,0,Platoon (1986),"[1, 11, 0, 0, 0, 0]",1,3,19,408
655568,3276,2003,1,965682094,"Palm Beach Story, The (1942)","[6, 0, 0, 0, 0, 0]",0,4,4,2104
...,...,...,...,...,...,...,...,...,...,...
950341,5272,426,1,960857360,Walkabout (1971),"[1, 0, 0, 0, 0, 0]",0,2,15,2202
1999038,5597,248,0,0,Nikita (La Femme Nikita) (1990),"[12, 0, 0, 0, 0, 0]",0,2,7,1152
307016,928,0,1,974730227,Toy Story (1995),"[2, 3, 6, 0, 0, 0]",1,3,16,459
970345,5387,3644,1,957783320,"Adventures of Robin Hood, The (1938)","[7, 8, 0, 0, 0, 0]",1,5,13,778
